In [1]:
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt
import math
import time
#相比于最开始拿到的版本，qiskit更新了几处表达，
#result.get_data（）改为result.data（），data的表达也有变化（可以print获得的data看看data不在表示为data['001']这样的形式）

In [2]:
# import math lib
from math import pi

# import Qiskit
from qiskit import Aer, execute#aer是模拟器
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# import basic plot tools
from qiskit.tools.visualization import plot_histogram

In [3]:
# To use local qasm simulator  模拟器改了 之前是qasm
backend = Aer.get_backend('aer_simulator')

In [4]:
def get_theta(d):
    i = 0
    theta = 0
    count = 0
    for i in d:
        count = i+count
   # print(d)
    theta = 2*math.acos(count/384)
   # print(theta)
    return theta

In [5]:
def get_Distance(x,y):
    theta_1 = get_theta(x)
    theta_2 = get_theta(y)
    
    # create Quantum Register called "qr" with 3 qubits
    qr = QuantumRegister(3, name="qr")
    # create Classical Register called "cr" with 3 bits
    cr = ClassicalRegister(1, name="cr")

    # Creating Quantum Circuit called "qc" involving your Quantum Register "qr"
    # and your Classical Register "cr"
    qc = QuantumCircuit(qr, cr, name="k_means")
    
    qc.h(qr[0])
    qc.h(qr[1])
    qc.h(qr[2])
    qc.u(theta_1, pi, pi, qr[1])
    qc.u(theta_2, pi, pi, qr[2])
    qc.cswap(qr[0], qr[1], qr[2])
    qc.h(qr[0])

    qc.measure(qr[0], cr[0])
    qc.reset(qr)

    #print('----before run----')
    job = execute(qc,backend=backend, shots=1024)
    #print('----after run----')
    result = job.result()
    data = result.data()['counts']
    #print(data)
    
    if len(data)==1:
        return 0.0
    else:
        return data['0x1']/1024.0

In [6]:
def get_data(dataset_name,model_name,data_num):
    import datasets
    dataset = datasets.load_from_disk(dataset_name)
    from sentence_transformers import SentenceTransformer 
    model = SentenceTransformer(model_name,cache_folder=r"D:\HF-model\sentence-t5-base")
    from random import sample 
    import random
    n = random.randint(1,999999)  
    random.seed(n)
    sentences = sample(dataset['test'][0]['sentences'],data_num) 
    random.seed(n) 
    labels =  sample(dataset['test'][0]['labels'],data_num)
    embeddings = model.encode(sentences)#
    points=embeddings
    #print(type(points))
    #初始化聚类中心
    np.random.seed(96)
    centroids = np.random.random([32,384])
    return points,centroids,sentences,labels

In [7]:
def get_distance(p1, p2):
    #与经典没变化
    return np.sqrt(np.sum((p1-p2)*(p1-p2)))

In [8]:
def find_nearest_neighbour(points,centroids):
    
    n = len(points)
    k = centroids.shape[0]
    centers = np.zeros(n)
    
    for i in range(n):
        min_dis = 10000
        ind = 0#（Internodal Distance 节点间距离？）
        for j in range(k):
            #若这里为get_distance(points[i,:],centroids[j,:])则为经典算法
            #这里只对最近邻计算做量子化处理。是否还有可能用量子计算机处理的地方？
            
            temp_dis = get_Distance(points[i,:],centroids[j,:])
            
            if temp_dis < min_dis:
                min_dis = temp_dis
                ind = j
        centers[i] = ind
    
    return centers

In [9]:
def find_centroids(points,centers):
    n = len(points)
    k = int(np.max(centers))+1
   
    centroids = np.zeros([k,384])
    
    for i in range(k):
        #print(points[centers==i])
        centroids[i,:] = np.average(points[centers==i])
        
    return centroids

In [10]:
def preprocess(points):
    from sklearn.preprocessing import MinMaxScaler 
    scaler = MinMaxScaler() 
    normalized_data = scaler.fit_transform(points)
    
    return points

In [11]:
"""
可选数据集：
'D:\HF dataset\mteb/arxiv-clustering-p2p'
'D:\HF dataset\mteb/arxiv-clustering-s2s'
'D:\HF dataset\mteb/biorxiv-clustering-p2p'
'D:\HF dataset\mteb/biorxiv-clustering-s2s'
'D:\HF dataset\mteb/medrxiv-clustering-s2s'
'D:\HF dataset\mteb/reddit-clustering'
'D:\HF dataset\mteb/reddit-clustering-p2p'
'D:\HF dataset\mteb/stackExchange-clustering'
'D:\HF dataset\mteb/stackExchange-clustering-p2p'
'D:\HF dataset\mteb/twentynewsgroups-clustering'
"""

"\n可选数据集：\n'D:\\HF dataset\\mteb/arxiv-clustering-p2p'\n'D:\\HF dataset\\mteb/arxiv-clustering-s2s'\n'D:\\HF dataset\\mteb/biorxiv-clustering-p2p'\n'D:\\HF dataset\\mteb/biorxiv-clustering-s2s'\n'D:\\HF dataset\\mteb/medrxiv-clustering-s2s'\n'D:\\HF dataset\\mteb/reddit-clustering'\n'D:\\HF dataset\\mteb/reddit-clustering-p2p'\n'D:\\HF dataset\\mteb/stackExchange-clustering'\n'D:\\HF dataset\\mteb/stackExchange-clustering-p2p'\n'D:\\HF dataset\\mteb/twentynewsgroups-clustering'\n"

In [12]:
#range中的t1：想自动化出多少结果（每一遍都是同样大小数据集，但是选择的数据不同）
#t2 为QKmeans算法中迭代次数

t1 = 20
t2 = 1

for qq in range(10):
    output = np.array([['v_measure','标签种类(k)']])
        #选择数据集的模块
    if qq==0:
        dataset_name = 'D:\HF dataset\mteb/biorxiv-clustering-p2p'
    if qq==1:
        dataset_name = 'D:\HF dataset\mteb/biorxiv-clustering-s2s'
    if qq==2:
        dataset_name = 'D:\HF dataset\mteb/medrxiv-clustering-s2s'
    if qq==3:
        dataset_name = 'D:\HF dataset\mteb/reddit-clustering'
    if qq==4:
        dataset_name = 'D:\HF dataset\mteb/reddit-clustering-p2p'
    if qq==5:
        dataset_name = 'D:\HF dataset\mteb/stackExchange-clustering'
    if qq==6:
        dataset_name = 'D:\HF dataset\mteb/stackExchange-clustering-p2p'
    if qq==7:
        dataset_name = 'D:\HF dataset\mteb/twentynewsgroups-clustering'
    if qq==8:
        dataset_name = 'D:\HF dataset\mteb/arxiv-clustering-p2p'
    if qq==9:
        dataset_name = 'D:\HF dataset\mteb/arxiv-clustering-s2s'
    
        
    #计时模块
    start_time = time.time()
    
    #QKmeans
    for n1 in range(t1):

        #设置超参数
        model_name = 'sentence-t5-base'
        data_num = 100
    
        points,centroids1,sentences,labels = get_data(dataset_name,model_name,data_num)       #dataset

        points = preprocess(points)                # Normalize dataset
        # run k-means algorithm
        for i1 in range(t2):
    
            centers = find_nearest_neighbour(points,centroids1)       # find nearest centers
            #print(centers)
            centroids = find_centroids(points,centers)               # find centroids
        
        #一共有多少种不同的标签    
        k=[0]*len(labels)
        count = 0
        #print(k)
        flag1 = 0
        for i in labels: 
            flag1 = 0 
            for j in k: 
                if j == i: 
                    flag1 = 0
                    break 
                else:  
                    flag1 = 1
            if flag1 == 1:
                for n in range(len(labels)):
                    if k[n] == 0:
                        k[n] = i
                        break
                    else:
                        continue
        for m in k:
            if m == 0:
                break
            else:
                count = count + 1   
        #print(len(k))
        #print(k)
        #count 是单个数字类型
        #print('一共有多少种不同的标签：',count)
    
        #评价部分
        from mteb.evaluation.evaluators import ClusteringEvaluator
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('sentence-t5-base',cache_folder=r"D:\HF-model\sentence-t5-base")
        labels = centers
        clusterer = ClusteringEvaluator(sentences=sentences, labels=labels)
        #result是dic类型
        result = clusterer(model)
        #print(result)
        output = np.insert(output,n1+1,[str(result['v_measure']),str(count)],axis=0)
    output = np.insert(output,0,[dataset_name[19:],str(data_num)],axis=0)

    #计时模块
    end_time = time.time()
    cost_time = end_time - start_time

    filename = '%s组重复 %s %s.csv'%(t1,dataset_name[19:],model_name)
    np.savetxt(filename,output,fmt='%s',delimiter=',')
    print('END******************************************************')
    print(cost_time)

d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change f

d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change f

END******************************************************
964.9289801120758


d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
d:\Tool\Anaconda3\envs\pytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change f

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2359296 bytes.

In [ ]:
""" 学习块（load_dataset实验 保存到本地并使用）  
为什么D:\HF dataset\mteb/arxiv-clustering-s2s   
mteb后面一定要用反斜杠？？？？？？？？？？不用就识别的是mtebarxiv-clustering-s2s这样的路径"""
#from datasets import load_dataset 
#dataset = load_dataset("mteb/arxiv-clustering-p2p")
#dataset.save_to_disk('D:\HF dataset\mteb/arxiv-clustering-p2p')
#import datasets 
#dataset = datasets.load_from_disk("D:\HF dataset\mteb/arxiv-clustering-p2p")


"""修改网上下载数据集的缓存地址"""
#from datasets import load_dataset
#dataset = load_dataset('mteb/arxiv-clustering-p2p', cache_dir="D:\HF .cache")

In [ ]:
from sentence_transformers import SentenceTransformer     
model = SentenceTransformer('bert-base-uncased')

In [34]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 31
    })
})


In [ ]:
"""#评价模块
from mteb.evaluation.evaluators import ClusteringEvaluator
# Create ClusteringEvaluator instance
evaluator = ClusteringEvaluator()
# Evaluate the clustering results
evaluation_result = evaluator.evaluate(centers) 
# Access evaluation metrics 
print("Adjusted Rand Index:", evaluation_result["adjusted_rand_index"]) 
print("Normalized Mutual Information:", evaluation_result["normalized_mutual_information"])
print("Homogeneity Score:", evaluation_result["homogeneity_score"])
print("Completeness Score:", evaluation_result["completeness_score"])
print("V-Measure Score:", evaluation_result["v_measure_score"])"""

In [ ]:
"""from mteb.evaluation.evaluators import ClusteringEvaluator 
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
# Generate synthetic data for clustering X,
np.random.seed(1)
X = np.zeros((90, 3))#90行3列 
#print(X) 
for i in range(X.shape[0]):   
    if (i%3==0):        
        x_dim = 2 + np.random.randn() # 围绕(2,3)作中心，随机散部点        
        y_dim = 3 + np.random.randn()        
        X[i, 0] = x_dim         
        X[i, 1] = y_dim        
        X[i, 2] = 0     
    elif (i%3==1):         
        x_dim = 4 + np.random.randn() # 围绕(4,5)作中心，随机散部点     
        y_dim = 5 + np.random.randn()      
        X[i, 0] = x_dim         
        X[i, 1] = y_dim        
        X[i, 2] = 1     
    else:        
        x_dim = 7 + np.random.randn() # 围绕(7,6)作中心，随机散部点        
        y_dim = 6 + np.random.randn()      
        X[i, 0] = x_dim       
        X[i, 1] = y_dim      
        X[i, 2] = 2        
#print(X)
# Cluster the data using K-means 
kmeans = KMeans(n_clusters=3, random_state=42)
y_pred = kmeans.fit_predict(X)
# Create ClusteringEvaluator instance
evaluator = ClusteringEvaluator()
# Evaluate the clustering results 
evaluation_result = evaluator.evaluate(y_pred)
# Access evaluation metrics  
print("Adjusted Rand Index:", evaluation_result["adjusted_rand_index"]) 
print("Normalized Mutual Information:", evaluation_result["normalized_mutual_information"])
print("Homogeneity Score:", evaluation_result["homogeneity_score"])
print("Completeness Score:", evaluation_result["completeness_score"]) 
print("V-Measure Score:", evaluation_result["v_measure_score"])"""

In [ ]:
"""#构建MTEB 能够使用的模型（其实不需要，只要用MTEB 中的ClusteringEvaluator就能解决问题） 
from mteb import AbsTaskRetrieval, DRESModel 
class MyModel(DRESModel):       
# Refer to the code of DRESModel for the methods to overwrite   
    pass   
assert AbsTaskRetrieval.is_dres_compatible(MyModel)"""